### Preparation

#### 1.1 Import library

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from forecast.class_Forecast_Model import Forecast_Model
from create_database.class_ETL import ETL 

pd.options.display.float_format = '{:,}'.format
pd.options.display.max_columns = 100


In [2]:
###########################################################################################################################################################################################################################################
##  What can triendt can do for you? let type: triendt and the dot like "triendt."    
##                                                                                                                                               
##  As you can see, triendt has 4 functions which are:                                                                                                                                                                               
##      1. triendt.extract_bigquery_data(start_date=, table_name= )                                                                                                                                                                  
##      2. triendt.extract_local_data(table_name=)                                                                                                                                                                                   
##      3. triendt.ingest_data_to_sqlite(data_ingest=, table_name=)                                                                                                                                                                  
##      4. triendt.auto_ingest_data(table_name=)                                                                                                                                                                                     
###########################################################################################################################################################################################################################################
## 1. function extract_bigquery_data
##    >> extract data from an sql file, which is saved in create_database folder.  If you want to extract another table, create a file contains sql script, save it with the table_name.sql and store it in create_database folder
##    >> start_date is the date that you want to view
##    >> table_name is the name of the file, you can think of a sql file like a metabase card withou filter anything.
## 
## 2. function extract_local_data
##    >> extract data from fa_database.db file, you can check it out at this link (https://sqliteviewer.app/) to see which table already exists in local db
##    >> table_name is the name of table already has data locally   
##
## 3. ingest_data_to_sqlite 
##    >> use to insert data local, avoid being updated lately.
##    >> table_name is the name of table already has data locally   
## Note: Carefully when use it funcion, because this function allow use insert data without check out whether new data to insert already exist in DB. As a result, data could be duplicated, and downstreaming purpose would be incorect.
##
## 4. auto_ingest_data
##    >> use this function to update data in database locally without being afraid of data duplicate
##    >> data will be updated with current date - 1 day. 
############################################################################################################################################################################################################################################

#### 1.2 Create udf - user defined function

In [2]:
def filter_data_input(df, frc_value, cond):
    
    # fitler your df
    if cond:
        for _key, _values in cond.items():
            df = df[df[_key] == _values ]
    
    df_pivot = df.groupby(['complete_date'],as_index = False)[frc_value].sum()
    df_pivot.sort_values('complete_date',inplace= True)

    return df_pivot

### Forecast

In [3]:
# call triendt to handle import, export data
triendt = ETL(credentials_file_path= './../_config/cred.json',
              local_database= './../_data_raw/fa_database.db'
              )

In [15]:
df_bigquery_data = triendt.extract_bigquery_data(start_date= '2024-04-01', table_name= 'fct_fa_pnl_mini')
df_bigquery_data

##### 2.1 Check if historical data exists 
* If your local database is not update data yet. Just use function auto_updat_data above

In [6]:
df_fa_pnl_mini = triendt.extract_local_data(table_name='fct_fa_pnl_mini')
df_fa_pnl_mini

In [5]:
## to delete all data and update whole table, choose update_type = 'update_as_new'
triendt.auto_update_data(table_name='fct_fa_pnl_mini', update_type='append')

##### 2.2 Input filter condition below and change your df name with condition.

In [6]:
filter_conditions = {"province": "han",
                     "service_type": "truck",
                     }

df_historical_data = filter_data_input(df= df_fa_pnl_mini,frc_value= 'gsv_excluded_vat', cond= filter_conditions)

print('''Your historical data:''')

df_historical_data.head()

In [7]:
### call forecast model
forecast_gsv_bike_sgn = Forecast_Model(historical_data=df_historical_data,
                                      date_column='complete_date',
                                      forecast_value= 'gsv_excluded_vat',
                                      )

### forecast for the next 365 days
df_forecast = forecast_gsv_bike_sgn.call_forecast_model(no_of_days_forecast= 365)
df_forecast

In [8]:
forecast_gsv_bike_sgn.resample_and_plot_data(df_forecast=df_forecast,start_date='2022-01-01')


### Export data
* call funtion export_data, provide dataframe, file type (excel or csv) and file name

In [9]:
# forecast_gsv_bike_sgn.export_data(df=df_forecast,file_name='gsv_bike_sgn',file_type='excel')
forecast_gsv_bike_sgn.export_data(df=df_forecast,file_name='gsv_truck_sgn',file_type='csv')